## Getting Polyline data from Studio and transferring to Geodatabase

With recent updates to Studio database the geometry type that is stored within the database was changed and ArcGIS products were no longer able to read it. There were work-arounds involving creating copies of query layers or other creative methods that added steps onto what use to be a strightforward method.


This notebook suggests one work around that leverages the geometry options within the arcgis library.
The basic run down: 

1. Write a query to the Studio database grabbing a table that has a shape field for a polyline
2. Pull out geometry information from SDO_object contained in shape field
3. Feed the geometry information to the Polyline function in arcgis library. This will make a true polyline geometry
4. Make every row into a Feature containing a dictionary of every row containing the attributes as well as the geometry 
5. Make a FeatureSet from a list of all the Features
6. Turn into spatial dataframe and save to file 

Originally created 1/22/2021  
---Noah Karsky---

Modified on 8/7/2021
---Jesus Avellaneda---


In [1]:
###IMPORT  LIBRARIES#####

from arcgis.geometry import Point, Polyline, Polygon, Geometry
import pandas as pd
import arcgis
from arcgis.features import SpatialDataFrame as sdf
from arcgis.features import Feature, FeatureSet, FeatureCollection


# these libraries would have to be downloaded from Nexus as I dont think they ship with ArcPro 2.5
import cx_Oracle
import sqlalchemy
from sqlalchemy import create_engine
import os
import arcpy



In [2]:
QUERY = '''SELECT UWI,
       BOREHOLE_ID,
       BOREHOLENAME,
       BULKARRAYID,
       ID,
       ISACTIVE,
       GUID,
       EXTENDEDPROPERTIES_,
       INSERT_DATE,
       LINE_,
       NAME,
       MD,
       TVD,
       FOLDER_PATH,
       SHAPE,
       UPDATE_DATE
  FROM (SELECT ROOT.BOREHOLE_ID BOREHOLE_ID,
               BOREHOLE.NAME BOREHOLENAME,
               ROOT.DEVIATION_SURVEY_BULK_DATA_ID BULKARRAYID,
               ROOT.REMARKS COMMENTS,
               ROOT.COORDINATE_SYSTEM_ID COORDINATESYSTEM_ID,
               CRS CRSCODE,
               EXTENDEDPROPERTIES.ID EXTENDEDPROPERTIES_,
               ROOT.GUID GUID,
               ROOT.ID ID,
               BOREHOLE.DRILLER_TD_MD MD,
               BOREHOLE.DRILLER_TD_TVD TVD,
               ROOT.INSERT_DATE INSERT_DATE,
               ROOT.INSERT_USER INSERT_USER,
               CASE
                  WHEN    DEFINITIVE_SURVEY_ID = ROOT.ID
                       OR ACTIVE_PLAN_ID = ROOT.ID
                  THEN
                     1
                  ELSE
                     0
               END
                  ISACTIVE,
               LINE.ID LINE_,
               ROOT.NAME NAME,
               ORIGINAL_COORD_SYSTEM_ID ORIGINALCOORDSYSTEM_ID,
               LINE.SHAPE SHAPE,
               PARENT_GUID PARENTGUID,
               CASE Root.Existence_Kind
                  WHEN 'Explicit' THEN 'Survey'
                  WHEN 'XYZ' THEN 'Survey'
                  WHEN 'MdIncAzi' THEN 'Survey'
                  WHEN 'XYTvd' THEN 'Survey'
                  WHEN 'DxDyTvd' THEN 'Survey'
                  WHEN 'AutoPlan' THEN 'Plan'
                  WHEN 'ExplicitPlan' THEN 'Plan'
                  WHEN 'XYZPlan' THEN 'Plan'
               END
                  SURVEYTYPE,
               ROOT.TIE_IN_MD TIEINMD,
               ROOT.PARENT_SURVEY_ID TIEINSURVEYPLAN_ID,
               PARENTSURVEYPLAN.NAME TIEINSURVEYPLANNAME,
               ROOT.INTERPRETATION_STATUS TRAJECTORYFORMAT,
               PROPERTIES TRAJECTORYPROPERTIES,
               ROOT.UPDATE_DATE UPDATE_DATE,
               ROOT.UPDATE_USER UPDATE_USER,
               SELF.Folder_Path,
               UBHI UWI
          FROM GUYANA_INT.DEVIATION_SURVEY_ ROOT
               INNER JOIN GUYANA_INT.COORDINATE_SYSTEM_ COORDINATESYSTEM
                  ON (ROOT.COORDINATE_SYSTEM_ID = COORDINATESYSTEM.ID)
               LEFT OUTER JOIN GUYANA_INT.DEVIATION_SURVEY_ PARENTSURVEYPLAN
                  ON (ROOT.PARENT_SURVEY_ID = PARENTSURVEYPLAN.ID)
               LEFT OUTER JOIN GUYANA_INT.DBX_OBJECT_ SELF
                  ON (SELF.ENTITY_ID = ROOT.ID)
               LEFT OUTER JOIN GUYANA_INT.LINE_ LINE
                  ON (LINE.ID = ROOT.SURVEY_TRAJECTORY_ID)
               LEFT OUTER JOIN
               GUYANA_INT.DBX_EXTENDED_PROPERTIES_ EXTENDEDPROPERTIES
                  ON (EXTENDEDPROPERTIES.ENTITY_ID = ROOT.ID)
               INNER JOIN GUYANA_INT.BOREHOLE_ BOREHOLE
                  ON (BOREHOLE.ID = ROOT.BOREHOLE_ID)) MAIN_VIEW
                  WHERE ISACTIVE=1 AND LINE_ IS NOT NULL
'''



In [3]:
##Making the engine that connects to Studio database
engine = create_engine('oracle://@P_PET_SAMP') # creating a connection to the studio database
connection = engine.connect()
print(engine)



Engine(oracle://@P_PET_SAMP)


In [4]:
##this first function was puled from  a stack overflow thread it allows us to pull apart the oracle obj

def ObjectRepr(obj):
    if obj.type.iscollection:
        returnValue = []
        for value in obj.aslist():
            if isinstance(value, cx_Oracle.Object):
                value = ObjectRepr(value)
            returnValue.append(value)
    else:
        returnValue = {}
        for attr in obj.type.attributes:
            value = getattr(obj, attr.name)
            if value is None:
                continue
            elif isinstance(value, cx_Oracle.Object):
                value = ObjectRepr(value)
            returnValue[attr.name] = value
    return returnValue


#the part we care about inside of the oracle object is the SDO_ORDINATES and in our example it has 4 parts to each point
#this function takes the big list of all the points and breaks it into many lists with 4 elements
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

#This function applies the above two functions and then removes the 4th element in every point as it is "none"
#then it forms a dictionary called line that has every point
#it then feeds the dictionary to the Polyline function
def make_geom(shape_item,):
    get_obj = ObjectRepr(shape_item)
    path_list = list(divide_chunks(get_obj['SDO_ORDINATES'],4))      #grabs SDO_ORDINATE and cunks it into lists of 4
    [x.pop() for x in path_list]         #remove 4th element from every point leaving x,y,z
    #builds a line dictionary using the path_list and assumes WGS84
    line = {
    "paths": [path_list],
    "spatialReference" : {"wkid" : 4326}}
    polyline = Polyline(line,hasZ=True)#feeds the line to Polyline making polyline function
    return polyline


def make_featureclass(dataframe):
    dataframe.dropna(subset=['shape'],inplace=True) #we drop rows with no geometry
    dataframe['SHAPE'] = dataframe['shape'].apply(lambda x: make_geom(x)) #takes prior function and applies it every row
    dataframe.drop('shape',axis=1,inplace=True)   #we no longer need the old shape
    inx_array = dataframe.index.values 
    feature_list = []
    for x in inx_array:
        ent1 = dataframe.iloc[x,-1]  #the attribute data for each row
        ent2 = dataframe.iloc[x,:-1].to_dict()    #the geometry data for each row
        feature = Feature(geometry=ent1,attributes=ent2) #making a feature from each row
        feature_list.append(feature)   #making a featureSet from each row
    return FeatureSet(features=feature_list)





In [5]:
df = pd.read_sql_query(sql = QUERY,con = connection) 

In the cell above if you scroll to the end you can see the Shape column. it is stored as a special type of cx_Oracle object

In [6]:
print(' this is what the Oracle object looks like: {} so we need a way to unpack it'.format(df.loc[45,'shape']))

 this is what the Oracle object looks like: <cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x1817ca50> so we need a way to unpack it


In [7]:
#use ObjRepair function on it
ObjectRepr(df.loc[45,'shape'])

{'SDO_GTYPE': 4402,
 'SDO_ELEM_INFO': [1, 2, 1],
 'SDO_ORDINATES': [-55.1538606000007,
  6.81581559999999,
  10,
  None,
  -55.1538606000007,
  6.81581559999999,
  -1923.6952,
  None]}

The output above are all the elements that were inside of the Oracle object, They all correspond to data that within the SDO geometry. One can understand what all these mean with a simple websearch, but we are only after the SDO_ORDINATES.

We are essentially given all the different points within a polygon, and we need to break it up into each point. 

To do this, we split it into a list of 4 elements (x,y,z,m) ***I think it is m*** This feature class has no m, so we end up popping it off to leave us with an  x,y,z that we then format for use in the arcgis library.

That really details the part that took me forever to find out. The rest of the code consists of formatting these coordinates into dictionaries to be used by the arcgis library. If you want to look into the code more all the formatting happens in the functions above

In [8]:
df = pd.read_sql_query(sql = QUERY,con = connection) 
feature_set = make_featureclass(df)

In [9]:
type(feature_set)

arcgis.features.feature.FeatureSet

At this point we have a feature set that has been loaded with all our data and we can turn it into a data frame

In [10]:
feature_sdf = feature_set.sdf

feature_sdf['top_md'] = 0

#to turn it into a feature class you would use the method below:
feature_sdf.spatial.to_featureclass(r"I:\UTCS\Region\SAmerica.Hou\Ref_Data\Guyana.hou\General\Maps\Guyana_INT_to_Guyana_SDE\Guyana_INT_to_Guyana_SDE.gdb\GUYANA_INT_WELL_PATHS",has_z=True,has_m=True)


'I:\\UTCS\\Region\\SAmerica.Hou\\Ref_Data\\Guyana.hou\\General\\Maps\\Guyana_INT_to_Guyana_SDE\\Guyana_INT_to_Guyana_SDE.gdb\\GUYANA_INT_WELL_PATHS'

In [11]:
feature_sdf

,uwi,borehole_id,boreholename,bulkarrayid,id,isactive,guid,extendedproperties_,insert_date,line_,name,md,tvd,folder_path,update_date,OBJECTID,SHAPE,top_md
0,100000839039,33703929,SORUBIM 1,33703935,33703940,1,9902ED4730FA508E9B07D181BD61B007,33703940,2021-06-09 07:50:03,33703940,Explicit survey 1,3822.700,3822.588963,\Project\Input\Wells\Guyana\Guyana Basin\SORUB...,2021-06-09 07:50:03,1,"{""hasZ"": true, ""paths"": [[[-57.7666670000001, ...",0
1,100000003421,285241,MAHAICA 2,32387020,32387018,1,6020EADDBA1C4B96913109A009707F77,32387018,2020-10-07 15:53:25,32387018,Two_Points_Survey,2286.000,2261.000000,\Project\Input\Wells\Guyana\Guyana Basin\MAHAI...,2020-10-07 15:53:25,2,"{""hasZ"": true, ""paths"": [[[-57.1581500000001, ...",0
2,21418000590,15428806,LIZ_2P1,32355205,32355111,1,FC5795BB552C4825B4B1F0009FAD34BD,32355111,2020-10-06 13:51:11,32355111,LIZ_2P1:20191216,7724.000,5281.265808,\Project\Input\Wells\Guyana\Guyana Basin\LIZ_2P1,2020-10-06 13:51:11,3,"{""hasZ"": true, ""paths"": [[[-56.9625728100001, ...",0
3,21418001300,27242856,LIZ_1G2,32402042,32402040,1,7C84B042F6EB4B6CB0ED927901DB5152,32402040,2020-10-14 08:28:28,32402040,LIZ_1G2:20200923,7012.000,5143.200519,\Project\Input\Wells\Guyana\Guyana Basin\LIZ_1G2,2020-10-14 08:28:28,4,"{""hasZ"": true, ""paths"": [[[-56.9661822200001, ...",0
4,100000003422,285242,MAHAICA 1,32387531,32387529,1,728AD9563FE9409F8C609C7AB4745CBA,32387529,2020-10-07 15:53:46,32387529,Two_Points_Survey,2451.000,2441.000000,\Project\Input\Wells\Guyana\Guyana Basin\MAHAI...,2020-10-07 15:53:46,5,"{""hasZ"": true, ""paths"": [[[-57.14732, 7.467659...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,100000860201,32084674,TUKUNARI 1,33923323,33923321,1,010F5FACB8674FA49DE52D9D7DB93ED5,33923321,2021-07-29 13:19:57,33923321,Two_Points_Survey,875.000,835.000000,\Project\Input\Wells\Suriname\Guyana Basin\TUK...,2021-07-29 13:19:57,1190,"{""hasZ"": true, ""paths"": [[[-55.6556360000002, ...",0
1190,21418001450,33961701,LIZ_5P5,33961705,33961703,1,8D65F2E9DD6A41C483274F4566BD0708,33961703,2021-08-06 13:08:43,33961703,Two_Points_Survey,0.001,-24.999000,\Project\Input\Wells\Guyana\Guyana Basin\LIZ_5...,2021-08-06 13:08:43,1191,"{""hasZ"": true, ""paths"": [[[-56.9202421356907, ...",0
1191,21418001070,33888595,Turbot-2,33972348,33972346,1,6C9C3B13BC4145D1B3BC7BAEA8123F1B,33972346,2021-08-12 09:08:30,33972346,Turbot-2:20210804,5740.000,5612.903524,\Project\Input\Wells\Guyana\Guyana Basin\Turbo...,2021-08-12 09:08:30,1192,"{""hasZ"": true, ""paths"": [[[-56.5293950000001, ...",0
1192,21418000530,13532618,Liza-5,33975226,33975224,1,7406301BF53F4AC68D32A322421BDB2E,33975224,2021-08-16 07:14:41,33975224,Liza-5:20180301,5643.000,5598.814852,\Project\Input\Wells\Guyana\Guyana Basin\Liza-...,2021-08-16 07:14:41,1193,"{""hasZ"": true, ""paths"": [[[-56.9234317200001, ...",0


In [12]:
feature_sdf.boreholename.sort_values().unique()

array(['8Q 3', '8V 13', '8X 19', ..., 'Yellowtail-1 [D2Z]',
       'Yellowtail-2', 'Yellowtail-2 ST1'], dtype=object)

## Part 2

We export our feature class and then create routes along it to get an M value. 

In [13]:
arcpy.env.workspace = r'I:\UTCS\Region\SAmerica.Hou\Ref_Data\Guyana.hou\General\Maps\Guyana_INT_to_Guyana_SDE\Guyana_INT_to_Guyana_SDE.gdb'
arcpy.env.overwriteOutput = True

fc_location = r'I:\UTCS\Region\SAmerica.Hou\Ref_Data\Guyana.hou\General\Maps\Guyana_INT_to_Guyana_SDE\Guyana_INT_to_Guyana_SDE.gdb\GUYANA_INT_WELL_PATHS'
fc_route_location = fc_location + "_SDE" #slap an _SDE on the end
arcpy.lr.CreateRoutes(fc_location, "guid", fc_route_location,  "TWO_FIELDS", "top_md", "md", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.JoinField_management(fc_route_location,'guid',fc_location,'guid')
arcpy.DeleteField_management(fc_route_location,['Shape_Length_1','guid_1'])

<Result 'I:\\UTCS\\Region\\SAmerica.Hou\\Ref_Data\\Guyana.hou\\General\\Maps\\Guyana_INT_to_Guyana_SDE\\Guyana_INT_to_Guyana_SDE.gdb\\GUYANA_INT_WELL_PATHS_SDE'>

In [14]:
SDE_well_path = r'I:\UTCS\Region\SAmerica.Hou\Ref_Data\Guyana.hou\General\Maps\Guyana_INT_to_Guyana_SDE\SDE_GUYANA.sde\SDE_GUYANA.GIS.GUYANA_INT_WELL_PATHS'


arcpy.DeleteRows_management(SDE_well_path)
#I just copied the line below from ArcPro
arcpy.management.Append(fc_route_location,SDE_well_path, "NO_TEST")

<Result 'I:\\UTCS\\Region\\SAmerica.Hou\\Ref_Data\\Guyana.hou\\General\\Maps\\Guyana_INT_to_Guyana_SDE\\SDE_GUYANA.sde\\SDE_GUYANA.GIS.GUYANA_INT_WELL_PATHS'>